# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [2]:
DATA_FOLDER = 'Data/'
import pandas as pd
import numpy as np
import glob

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
EBOLA_FOLDER = DATA_FOLDER + 'ebola/'

In [ ]:
file_gui = glob.glob(EBOLA_FOLDER + "guinea_data/*.csv")
file_lib = glob.glob(EBOLA_FOLDER + "liberia_data/*.csv")
file_sie = glob.glob(EBOLA_FOLDER + "sl_data/*.csv")
list_gui = []
#Create DataFrame for Guinea
for file in file_gui:
    df = pd.read_csv(file,header=0,index_col=None,parse_dates=['Date'])
    list_gui.append(df)
ebola_gui = pd.concat(list_gui)

list_lib = []
#Create DataFrame for Liberia
for file in file_lib:
    df = pd.read_csv(file,header=0,index_col=None,parse_dates=['Date'])
    list_lib.append(df)
ebola_lib = pd.concat(list_lib)

list_sie = []
#Create DataFrame for Sierra Leone
for file in file_sie:
    df = pd.read_csv(file,header=0,index_col=None,parse_dates=['date'])
    list_sie.append(df)
ebola_sie = pd.concat(list_sie)

In [ ]:
res = ebola_gui.where(ebola_gui['Description'] == 'New cases of confirmed')
df = res[pd.notnull(res['Description'])]
new_cases_confirmed = df[['Totals', 'Date']]
new_cases_confirmed = new_cases_confirmed.rename(columns = {'Totals': 'New cases'})

res = ebola_gui.where(ebola_gui['Description'] == 'New deaths registered')
df = res[pd.notnull(res['Description'])]
new_deaths_registered = df[['Totals', 'Date']]
new_deaths_registered = new_deaths_registered.rename(columns = {'Totals': 'New deaths'})
test = new_cases_confirmed.merge(new_deaths_registered, on='Date')
test['Country'] = 'Guinea'
test = test.rename(columns={'Date': 'Month'})
test['Month'] = test.Month.dt.strftime("%B")
test

In [ ]:
res = ebola_lib.where(ebola_lib['Variable'] == 'New case/s (confirmed)')
df = res[pd.notnull(res['Variable'])]
new_cases_confirmed = df[['National', 'Date']]
new_cases_confirmed = new_cases_confirmed.rename(columns = {'National': 'New cases'})
new_cases_confirmed

In [ ]:
res = ebola_lib.where(ebola_lib['Variable'] == 'New case/s (confirmed)')
df = res[pd.notnull(res['Variable'])]
new_cases_confirmed = df[['National', 'Date']]
new_cases_confirmed = new_cases_confirmed.rename(columns = {'National': 'New cases'})

res = ebola_lib.where(ebola_lib['Variable'] == 'Newly reported deaths')
df = res[pd.notnull(res['Variable'])]
new_deaths_registered = df[['National', 'Date']]
new_deaths_registered = new_deaths_registered.rename(columns = {'National': 'New deaths'})
test = new_cases_confirmed.merge(new_deaths_registered, on='Date')
test['Country'] = 'Liberia'
test = test.rename(columns={'Date': 'Month'})
test['Month'] = test.Month.dt.strftime("%B")
test

In [ ]:
ebola_lib[list(['Date','Variable','National'])]

In [ ]:
# We are using 'etc_new_deaths' for computing the number of deaths per day because it's the only information we have.
# 'etc' means 'Ebola Treatment Center', so those seem to be the deaths happening in the treatment center.

res = ebola_sie.where(ebola_sie['variable'] == 'new_confirmed')
df = res[pd.notnull(res['variable'])]
new_cases_confirmed = df[['National', 'date']]
new_cases_confirmed = new_cases_confirmed.rename(columns = {'National': 'New cases'})
#mean = np.ceil(np.mean(list(map(int, new_cases_confirmed['New cases'].dropna().values))))
#new_cases_confirmed = new_cases_confirmed.fillna(mean)

res = ebola_sie.where(ebola_sie['variable'] == 'etc_new_deaths')
df = res[pd.notnull(res['variable'])]
new_deaths_registered = df[['National', 'date']]
new_deaths_registered = new_deaths_registered.rename(columns = {'National': 'New deaths'})
#mean = np.ceil(np.mean(list(map(int, new_deaths_registered['New deaths'].dropna().values))))
#new_deaths_registered = new_deaths_registered.fillna(mean)
total = new_cases_confirmed.merge(new_deaths_registered, on='date')
total['Country'] = 'Sierra Leone'
total = total.rename(columns={'date': 'Month'})
total['Month'] = total.Month.dt.strftime("%B")
total

In [ ]:
ebola_sie[list(['date','variable','National'])]

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

## Solution

### Explanation

Here we have 9 spreedsheets containing microbiome data. This data can be seen as a single table containing 2 columns, the `NAME` and the `COUNT`. Thus, we simply read those files and concatenate them into a single dataframe representing our table with 2 columns.

Given that the file does not contain headers, we ignore the first row (which is not a header) and give a name to the columns with the `header=None` and `names=['NAME', 'COUNT']` parameters.

We also add a new `BARCODE` column to the dataframe, which is the name corresponding to the column name in the metadata file. We give it the same name in order to have normalized dataframes that can be merged on columns with the same name.

Finally, we read the metadata file into a dataframe that we merge with our initial dataframe, we replace NA values with the `unknown` value and place an index on `[BARCODE, NAME]` in order to have unique indexes

### Corresponding Code

In [95]:
MICROBIOME_FOLDER = DATA_FOLDER + 'microbiome/'
mid_files = glob.glob(MICROBIOME_FOLDER + "MID*.xls")

In [96]:
# Initial empty dataframe
mid = pd.DataFrame()
for idx, file in enumerate(mid_files, 1):
    file_name = 'MID' + str(idx)
    # We give appropriate names to colummns and replace NA values with "unknown"
    tmp_mid = pd.read_excel(MICROBIOME_FOLDER+file_name+'.xls', names=['NAME', 'COUNT'], header=None)
    # We add a column "BARCODE" corresponding to the name of the file being read
    tmp_mid['BARCODE'] = file_name
    # We concatenate the values to create a single dataframe
    mid = pd.concat([mid, tmp_mid])

# We read the metadata file and merge it to the MID dataframe where columns have the same name
metadata = pd.read_excel(MICROBIOME_FOLDER + 'metadata.xls')
# We create an unique index on BARCODE and TAXON and replace NA values by 'unknown'
final_df = pd.merge(mid, metadata).fillna('unknown').set_index(['BARCODE', 'NAME'])
final_df

COUNT  \
BARCODE NAME                                                        
MID1    Archaea "Crenarchaeota" Thermoprotei Desulfuroc...      7   
        Archaea "Crenarchaeota" Thermoprotei Desulfuroc...      2   
        Archaea "Crenarchaeota" Thermoprotei Sulfolobal...      3   
        Archaea "Crenarchaeota" Thermoprotei Thermoprot...      3   
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...      7   
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...      1   
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...      1   
        Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...      1   
        Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...      1   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      1   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      4   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      1   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      2   
        Archaea "Euryarchaeota" Halobacteria Halobacter...      4   
        Archaea "Euryarchaeota" Methanococci Methanococ...      1   
        Archaea "Euryarchaeota" Methanopyri Methanopyra...     12   
        Archaea "Euryarchaeota" Thermoplasmata Thermopl...      2   
        Bacteria "Actinobacteria" Actinobacteria Actino...      2   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...      2   
        Bacteria "Actinobacteria" Actinobacteria Actino...     53   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...      2   
        Bacteria "Actinobacteria" Actinobacteria Actino...      3   
        Bacteria "Actinobacteria" Actinobacteria Actino...      1   
        Bacteria "Actinobacteria" Actinobacteria Actino...     99   
        Bacteria "Actinobacteria" Actinobacteria Actino...     17   
        Bacteria "Actinobacteria" Actinobacteria Actino...      2   
...                                                           ...   
MID9    Bacteria "Proteobacteria" Gammaproteobacteria O...     10   
        Bacteria "Proteobacteria" Gammaproteobacteria O...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria O...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria P...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria P...     12   
        Bacteria "Proteobacteria" Gammaproteobacteria P...      4   
        Bacteria "Proteobacteria" Gammaproteobacteria P...    805   
        Bacteria "Proteobacteria" Gammaproteobacteria P...    862   
        Bacteria "Proteobacteria" Gammaproteobacteria P...     12   
        Bacteria "Proteobacteria" Gammaproteobacteria P...   3534   
        Bacteria "Proteobacteria" Gammaproteobacteria T...      4   
        Bacteria "Proteobacteria" Gammaproteobacteria T...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria T...     25   
        Bacteria "Proteobacteria" Gammaproteobacteria T...      2   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      3   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      2   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      1   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      3   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      2   
        Bacteria "Proteobacteria" Gammaproteobacteria X...      1   
        Bacteria "Synergistetes" Synergistia Synergista...      3   
        Bacteria "Tenericutes" Mollicutes Anaeroplasmat...      1   
        Bacteria "Tenericutes" Mollicutes Haloplasmatal...     11   
        Bacteria "Thermodesulfobacteria" Thermodesulfob...      1   
        Bacteria "Thermodesulfobacteria" Thermodesulfob...      1   
        Bacteria "Thermotogae" Thermotogae Thermotogale...

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here